# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.project.domain.events.ProjectEvents.ProjectEvents import ProjectCreatedEvent
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x14b1ea600>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x14b4af470>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x14b1ea600>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x14b345cd0>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x14b1ea600>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.common.types.identity.Identity import UserId
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.Project import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    user_id=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 37 events
"Project Created" occurred at 2026-02-14T08:38:00.791836-03:00
"Project Created" occurred at 2026-02-14T08:38:27.557895-03:00
"Project Created" occurred at 2026-02-14T08:38:27.564485-03:00
"Project Created" occurred at 2026-02-14T08:38:27.567537-03:00
"Project Created" occurred at 2026-02-14T08:38:27.570584-03:00
"Project Created" occurred at 2026-02-14T08:38:27.572996-03:00
"Project Created" occurred at 2026-02-14T08:38:27.574673-03:00
"Project Created" occurred at 2026-02-14T08:38:27.576219-03:00
"Project Created" occurred at 2026-02-14T08:38:27.578161-03:00
"Project Created" occurred at 2026-02-14T08:38:27.580330-03:00
"Project Created" occurred at 2026-02-14T08:38:27.582370-03:00
"Project Created" occurred at 2026-02-14T08:38:41.894066-03:00
"Project Created" occurred at 2026-02-14T09:30:27.278290-03:00
"Project Created" occurred at 2026-02-15T13:40:44.867193-03:00
"Project Created" occurred at 2026-02-15T13:41:18.786000-03:00
"Project Created" occurred at 2026-02-1

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries


------------------------------


Found 37 events


Reset projection


Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 24 project summaries
Project 3ec7a4df-3903-49e9-94f8-3ee1b9933061: Rio Primero 123
Project 51319059-32c2-4e49-8ef6-1b709d8097ed: Project 0
Project 7b74841f-e7e4-439c-b382-d8c38830cc66: Project 1
Project 9c5bab18-7d8d-4a50-a215-0f64b2154901: Project 2
Project 1ae9e1f5-bd12-475a-8be0-0b3146f00f03: Project 3
Project 333b4969-dae1-4766-b65d-05816e45285d: Project 4
Project ce4b0a84-4295-42a9-805a-8b687cbfecf3: Project 5
Project 6ea64805-f0df-41b1-a801-08dc4ddfe736: Project 6
Project a97d417b-1157-4c02-84a4-98aec6d6b6f6: Project 7
Project f7bbba6d-0889-455b-83dd-265fa61dd51a: Project 8
Project be9b86f3-e4e8-4a18-b137-f072c50e102a: Project 9
Project 3a14ee9a-148a-4062-84cc-6e5c3e5c8baa: Project 1
Project 360931be-0df3-414d-af82-05475238d99e: Project 1
Project 84db37f6-80ec-4fce-bd8e-4752b65991ff: Rio Primero 123
Project f10a9cfc-8abf-428b-835c-343822166f82: Project 0
Project 4c241b3f-547a-4dfd-ab5d-60771470ba70: Project 1
Project 8f970df5-6ece-4241-9ef0-10cc0a0d229d: Project 2
Project 8